In [ ]:
import numpy as np

from mqt.qecc.se_synthesis import SyndromeExtractionEncoder, quasi_cyclic_check_matrices

In [ ]:
# Check matrices for the Steane Code
steane_x_checks = np.array([[1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1]])
steane_z_checks = steane_x_checks.copy()

# Decoding Circuit for the Steane Code

In [ ]:
encoder = SyndromeExtractionEncoder(steane_x_checks, steane_z_checks, 6)
res = encoder.solve()
encoder.get_circuit().draw(output="mpl")

# Decoder Circuits for Quasi-Cyclic Codes

In [ ]:
# Near Depth-optimal circuit
# This should be fast
x, z = quasi_cyclic_check_matrices(72)
encoder = SyndromeExtractionEncoder(x, z, 8)

res = encoder.solve()
encoder.get_circuit().depth()

In [ ]:
# ODepth-optimal Circuit
# WARNING: This will take a while
x, z = quasi_cyclic_check_matrices(72)
encoder = SyndromeExtractionEncoder(x, z, 7)

res = encoder.solve()
encoder.get_circuit().depth()